In [ ]:
# |default_exp models.general

In [ ]:
# |export
import torch as torch
import torch.nn as nn
from math import sqrt
import torch.nn.functional as F

In [ ]:
# |export
from torch.nn.modules import ReLU


class ExpertsBase(nn.Module):
    def __init__(self, n_exp, in_dim, out_dim, h_dim, act=nn.ReLU()):
        super().__init__()
        self.n_exp = n_exp

        def uniform(shape, scale):
            return nn.Parameter(torch.empty(shape).uniform_(-scale, scale))
        self.w1 = uniform((self.n_exp, h_dim, in_dim), scale=1 / sqrt(in_dim))
        self.w2 = uniform((self.n_exp, out_dim, h_dim), scale=1 / sqrt(h_dim))
        self.act = act


class TopkExperts(ExpertsBase):
    def forward(self, x: torch.Tensor, probs: torch.Tensor = None, indices: torch.Tensor = None):
        if indices is None: probs, indices = torch.topk(probs, self.n_exp)
        elif len(indices.shape) == 1: indices = indices[:, None]
        x = torch.einsum('bx,bkyx -> bky', x, self.w1[indices])
        x = torch.einsum('bkx,bkyx -> bky', self.act(x), self.w2[indices])
        if probs is None: return x.squeeze(1)
        return torch.einsum('bky,bk -> by', x, probs)


class SoftMergingExperts(ExpertsBase):
    def forward(self, x: torch.Tensor, probs: torch.Tensor = None, indices: torch.Tensor = None):
        """At least one of `probs` or `indices` should be not None."""
        bs = x.shape[0]
        if self.training:
            # construct a merged expert in the first layer
            w1 = torch.einsum('bkxy,bk->bxy', self.w1.repeat(bs, 1, 1, 1), probs)
            x = x[:, None].bmm(w1.mT).squeeze(1)

            x = self.act(x)

            # similar for the next layer
            w2 = torch.einsum('bkxy,bk->bxy', self.w2.repeat(bs, 1, 1, 1), probs)
            x = x[:, None].bmm(w2.mT).squeeze(1)
        else:
            w1 = self.w1[indices]
            x = x[:, None].bmm(w1.mT).squeeze(1)

            x = self.act(x)

            w2 = self.w2[indices]
            x = x[:, None].bmm(w2.mT).squeeze(1)
        return x

In [ ]:
# |export
class GateMoE(nn.Module):
    def __init__(self, in_dim, n_exp, top_k=1):
        super().__init__()
        self.n_exp, self.top_k = n_exp, top_k
        self.gate = nn.Linear(in_dim, n_exp, bias=False)

    def forward(self, x):
        probs, indices = torch.topk(self.gate(x), self.top_k, dim=-1)
        probs = F.softmax(probs, dim=-1)
        return probs, indices


class GateMoESoft(nn.Module):
    def __init__(self, in_dim, n_exp):
        super().__init__()
        self.n_exp = n_exp
        self.gate = nn.Linear(in_dim, n_exp, bias=False)

    def forward(self, x):
        if self.training:
            return F.softmax(self.gate(x), dim=-1), None
        return None, torch.argmax(self.gate(x), dim=-1)


class GateFFF(nn.Module):
    def __init__(self, in_dim, depth, tree_act=nn.Softplus()):
        super().__init__()
        self.tree_act, self.depth = tree_act, depth
        self.n = 2**depth

        def uniform(shape, scale):
            return nn.Parameter(torch.empty(shape).uniform_(-scale, scale))
        self.nodes = uniform((self.n - 1, in_dim), scale=1 / sqrt(in_dim))
        self.t = self.init_t_()

    def init_t_(self):
        tree, res = torch.eye(self.n), []
        for _ in range(self.depth):
            res.append(tree)
            tree = tree.view(self.n, -1, 2).sum(-1)
        return nn.Parameter(torch.cat(list(reversed(res)), dim=1), False)

    def forward(self, x):
        bs = x.shape[0]
        probs, indices = None, None
        if self.training:
            # get probs for each node
            x = x.matmul(self.nodes.T)
            probs = self.tree_act(torch.stack([x, -x], dim=2).view(bs, -1)).matmul(self.t.T)
            probs = torch.softmax(probs, dim=-1)
        else:
            # select node with highest prob at each level
            indices = torch.zeros(bs, dtype=torch.long, device=x.device)
            for _ in range(self.depth):
                indices = indices * 2 + 1 + (torch.einsum("bi,bi->b", x, self.nodes[indices]) < 0).long()
            # map to leaves that range from 0
            indices = indices - self.n + 1
        return probs, indices

In [ ]:
# |export
class GeneralMoE(nn.Module):
    def __init__(self, gate: nn.Module, experts: ExpertsBase, save_probs=False):
        super().__init__()
        self.save_probs = save_probs
        self.gate, self.experts = gate, experts

    def forward(self, x: torch.Tensor):
        probs, indices = self.gate(x)
        if self.save_probs:
            if probs is not None and probs.shape[-1]!=1: self.probs = probs.detach()
            else: self.probs = F.one_hot(indices.detach(), self.experts.n_exp)
        return self.experts(x, probs, indices)

In [ ]:
def moe(in_dim, out_dim, d, act=nn.GELU()):
    return GeneralMoE(GateMoE(in_dim, 2**d),
               TopkExperts(2**d, in_dim, out_dim, d, act))

In [ ]:
model = GeneralMoE(GateFFF(256, 3), TopkExperts(2**3, 256, 256, 10), save_probs=True)

In [ ]:
x = torch.randn(32, 256)

In [ ]:
model.eval()
model(x)

tensor([[ 0.1024, -0.0772,  0.1110,  ...,  0.1453,  0.3045, -0.1895],
        [ 0.1126, -0.1010, -0.1291,  ...,  0.1670, -0.0634, -0.0998],
        [-0.1895, -0.1292,  0.1696,  ..., -0.2021, -0.2963,  0.3774],
        ...,
        [ 0.3549, -0.3767, -0.2948,  ...,  0.2112,  0.0658, -0.2372],
        [-0.1207, -0.3709,  0.2090,  ..., -0.0537, -0.0338,  0.1172],
        [-0.2081,  0.0433,  0.0161,  ..., -0.1541,  0.0749,  0.0009]],
       grad_fn=<SqueezeBackward1>)